In [1]:
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, TensorDataset, Subset

from utils import *
from datasets import *
from mdav import *
from train import *
from models import *
from attacks import *

import random
import time
import copy
from collections import Counter
import csv

from sklearn.preprocessing import StandardScaler, LabelEncoder

In [2]:
import warnings
from sklearn.exceptions import ConvergenceWarning, FitFailedWarning

# Filter out ConvergenceWarning and FitFailedWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)
warnings.filterwarnings("ignore", category=FitFailedWarning)
warnings.filterwarnings("ignore", category=UserWarning)


In [3]:
# def seed_everything(seed=7):
#     np.random.seed(seed)
#     np.random.seed(seed)
#     random.seed(seed)
#     torch.manual_seed(seed)
#     torch.cuda.manual_seed(seed)
#     torch.backends.cudnn.deterministic = True
    
# seed_everything(seed=7)

In [ ]:
# Step 1: Get dataset

df=pd.read_csv('data/GiveMeSomeCredit/cs-training.csv')
df.drop(columns=['Unnamed: 0'], inplace=True)
df.dropna(inplace=True)
y = df['SeriousDlqin2yrs'].values
df.drop(df[['SeriousDlqin2yrs']],axis=1,inplace=True)
X = df.values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

SC = StandardScaler()
X_train = SC.fit_transform(X_train)
X_test = SC.transform(X_test)

# Sharding data
forget_ratio = 0.8
# Divide X_train into 5 equal shards
num_shards = 5
shard_size = len(y_train) // num_shards
X_shards = []
y_shards = []
retain_sets_X = []
retain_sets_y = []
forget_sets_X = []
forget_sets_y = []

for i in range(num_shards):
    # Calculate indices for slicing
    start_idx = i * shard_size
    end_idx = start_idx + shard_size if i < num_shards - 1 else len(y_train)
    
    # Slice the data to create shards
    X_shard = X_train[start_idx:end_idx]
    y_shard = y_train[start_idx:end_idx]
    
    X_shards.append(X_shard)
    y_shards.append(y_shard)
    
    # Shuffle indices for random sampling
    idxs = np.arange(len(y_shard))
    random.shuffle(idxs)
    m = int(len(y_shard) * forget_ratio)  # 5% forget ratio
    
    # Split indices for retain and forget sets
    retain_idxs = idxs[m:]
    forget_idxs = idxs[:m]
    
    # Create retain and forget sets for the shard
    X_retain = X_shard[retain_idxs]
    y_retain = y_shard[retain_idxs]
    X_forget = X_shard[forget_idxs]
    y_forget = y_shard[forget_idxs]
    
    retain_sets_X.append(X_retain)
    retain_sets_y.append(y_retain)
    forget_sets_X.append(X_forget)
    forget_sets_y.append(y_forget)


# Convert each shard's train, retain and forget sets into TensorDatasets
train_datasets = [
    TensorDataset(torch.tensor(X, dtype=torch.float32), torch.tensor(y, dtype=torch.int64))
    for X, y in zip(X_shards, y_shards)
]


retain_datasets = [
    TensorDataset(torch.tensor(X, dtype=torch.float32), torch.tensor(y, dtype=torch.int64))
    for X, y in zip(retain_sets_X, retain_sets_y)
]

forget_datasets = [
    TensorDataset(torch.tensor(X, dtype=torch.float32), torch.tensor(y, dtype=torch.int64))
    for X, y in zip(forget_sets_X, forget_sets_y)
]

batch_size = 512

# Create DataLoader instances for each retain and forget dataset
train_loaders = [DataLoader(dataset, batch_size=batch_size, shuffle=True) for dataset in train_datasets]
retain_loaders = [DataLoader(dataset, batch_size=batch_size, shuffle=True) for dataset in retain_datasets]
forget_loaders = [DataLoader(dataset, batch_size=batch_size, shuffle=False) for dataset in forget_datasets]

# Create TensorDatasets
train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.int64))
test_dataset = TensorDataset(torch.tensor(X_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.int64))

retain_dataset = torch.utils.data.ConcatDataset(retain_datasets)
forget_dataset = torch.utils.data.ConcatDataset(forget_datasets)

# Create DataLoader instances
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
retain_loader = DataLoader(retain_dataset, batch_size=batch_size, shuffle=True)
forget_loader = DataLoader(forget_dataset, batch_size=batch_size, shuffle=False)

num_features = X_train.shape[-1]
num_classes = len(set(y_train))

counter = Counter(y_train)
for k,v in counter.items():
    per = v / len(y_train) * 100
    print('Class=%s, Count=%d, Percentage=%.2f%%' % (k, v, per))
    
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
initial_model = MLPModel(num_features, 256, num_classes)
criterion = nn.CrossEntropyLoss()
lr = 1e-3
n_repeat = 3
max_epochs = 200
patience = None

In [ ]:
# Step 2: Define and train M on D
train_accs = []
test_accs = []
mia_aucs = []
mia_advs = []
runtimes = []
for r in range(n_repeat):
    torch.cuda.empty_cache()
    models = []
    t0 = time.time()
    for i in range(num_shards):
        model = copy.deepcopy(initial_model)
        optimizer = optim.Adam(model.parameters(), lr=lr)
        model = train_model(model, train_loaders[i], test_loader, criterion, optimizer, 
                            max_epochs, device=device, verbose_epoch = int(max_epochs/10), 
                            patience = patience)
        models.append(model)
    t1 = time.time()
    rt = t1-t0
    runtimes.append(rt)
    
    # Evaluate the model accuracy, and MIA
    # Accuracy
    train_acc = auc_score_with_majority_voting(models, train_loader)
    test_acc = auc_score_with_majority_voting(models, test_loader)
    train_accs.append(100.0*train_acc)
    test_accs.append(100.0*test_acc)
    #MIA
    idxs = np.arange(len(test_dataset))
    random.shuffle(idxs)
    m = len(forget_dataset)
    rand_idxs = idxs[:m]
    logits_test, loss_test, test_labels = compute_attack_components_sisa1(models, test_loader)
    logits_forget, loss_forget, forget_labels = compute_attack_components_sisa2(models, forget_loaders)
    attack_result = tf_attack(logits_forget, logits_test[rand_idxs], loss_forget, loss_test[rand_idxs], 
                          forget_labels, test_labels[rand_idxs])
    auc = attack_result.get_result_with_max_auc().get_auc()
    adv = attack_result.get_result_with_max_attacker_advantage().get_attacker_advantage()
    mia_aucs.append(100.0*auc)
    mia_advs.append(100.0*adv)

mean_runtime = np.mean(runtimes)
std_runtime = np.std(runtimes)
mean_train_acc = np.mean(train_accs)
std_train_acc = np.std(train_accs)
mean_test_acc = np.mean(test_accs)
std_test_acc = np.std(test_accs)
mean_mia_auc = np.mean(mia_aucs)
std_mia_auc = np.std(mia_aucs)
mean_mia_adv = np.mean(mia_advs)
std_mia_adv = np.std(mia_advs)

# Print the results
print('Training M on D time:{:0.2f}(±{:0.2f}) seconds'.format(mean_runtime, std_runtime))
print('Train accuracy:{:0.2f}(±{:0.2f})%'.format(mean_train_acc, std_train_acc))
print('Test accuracy:{:0.2f}(±{:0.2f})%'.format(mean_test_acc, std_test_acc))
print('MIA AUC:{:0.2f}(±{:0.2f})%'.format(mean_mia_auc, std_mia_auc))
print('MIA Advantage:{:0.2f}(±{:0.2f})%'.format(mean_mia_adv, std_mia_adv))

# Save to CSV
csv_file_path = 'results/SISA/credit/mlp_shards={}_fr={}_base.csv'.format(num_shards, forget_ratio)

with open(csv_file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Metric', 'Mean', 'Standard Deviation'])
    writer.writerow(['Training Time', mean_runtime, std_runtime])
    writer.writerow(['Train AUC', mean_train_acc, std_train_acc])
    writer.writerow(['Test AUC', mean_test_acc, std_test_acc])
    writer.writerow(['MIA AUC', mean_mia_auc, std_mia_auc])
    writer.writerow(['MIA Advantage', mean_mia_adv, std_mia_adv])

del models

In [ ]:
# Step 2: Retrain 
retain_accs = []
test_accs = []
mia_aucs = []
mia_advs = []
runtimes = []
for r in range(n_repeat):
    torch.cuda.empty_cache()
    models = []
    t0 = time.time()
    for i in range(num_shards):
        model = copy.deepcopy(initial_model)
        optimizer = optim.Adam(model.parameters(), lr=lr)
        model = train_model(model, retain_loaders[i], test_loader, criterion, optimizer, 
                            max_epochs, device=device, verbose_epoch = int(max_epochs/10), 
                            patience = patience)
        models.append(model)
    t1 = time.time()
    rt = t1-t0
    runtimes.append(rt)
    
    # Evaluate the model accuracy, and MIA
    # Accuracy
    retain_acc = auc_score_with_majority_voting(models, retain_loader)
    test_acc = auc_score_with_majority_voting(models, test_loader)
    retain_accs.append(100.0*retain_acc)
    test_accs.append(100.0*test_acc)
    #MIA
    idxs = np.arange(len(test_dataset))
    random.shuffle(idxs)
    m = len(forget_dataset)
    rand_idxs = idxs[:m]
    logits_test, loss_test, test_labels = compute_attack_components_sisa1(models, test_loader)
    logits_forget, loss_forget, forget_labels = compute_attack_components_sisa1(models, forget_loader)
    attack_result = tf_attack(logits_forget, logits_test[rand_idxs], loss_forget, loss_test[rand_idxs], 
                          forget_labels, test_labels[rand_idxs])
    auc = attack_result.get_result_with_max_auc().get_auc()
    adv = attack_result.get_result_with_max_attacker_advantage().get_attacker_advantage()
    mia_aucs.append(100.0*auc)
    mia_advs.append(100.0*adv)

mean_runtime = np.mean(runtimes)
std_runtime = np.std(runtimes)
mean_train_acc = np.mean(train_accs)
std_train_acc = np.std(train_accs)
mean_test_acc = np.mean(test_accs)
std_test_acc = np.std(test_accs)
mean_mia_auc = np.mean(mia_aucs)
std_mia_auc = np.std(mia_aucs)
mean_mia_adv = np.mean(mia_advs)
std_mia_adv = np.std(mia_advs)

# Print the results
print('Training M on D time:{:0.2f}(±{:0.2f}) seconds'.format(mean_runtime, std_runtime))
print('Train accuracy:{:0.2f}(±{:0.2f})%'.format(mean_train_acc, std_train_acc))
print('Test accuracy:{:0.2f}(±{:0.2f})%'.format(mean_test_acc, std_test_acc))
print('MIA AUC:{:0.2f}(±{:0.2f})%'.format(mean_mia_auc, std_mia_auc))
print('MIA Advantage:{:0.2f}(±{:0.2f})%'.format(mean_mia_adv, std_mia_adv))

# Save to CSV
csv_file_path = 'results/SISA/credit/mlp_shards={}_fr={}_retrain.csv'.format(num_shards, forget_ratio)

with open(csv_file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Metric', 'Mean', 'Standard Deviation'])
    writer.writerow(['Training Time', mean_runtime, std_runtime])
    writer.writerow(['Train AUC', mean_train_acc, std_train_acc])
    writer.writerow(['Test AUC', mean_test_acc, std_test_acc])
    writer.writerow(['MIA AUC', mean_mia_auc, std_mia_auc])
    writer.writerow(['MIA Advantage', mean_mia_adv, std_mia_adv])
